<a href="https://colab.research.google.com/github/ProfSynapse/Toolset-Training/blob/main/kto_colab_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Package Installation and Imports
Install required packages including unsloth and flash-attention, and import necessary libraries for the KTO finetuning process.

In [ ]:
# Install required packages
%%capture
!pip install unsloth
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git

# Install Flash Attention 2 for softcapping support
import torch
if torch.cuda.get_device_capability()[0] >= 8:
    !pip install --no-deps packaging ninja einops "flash-attn>=2.6.3"

# Import necessary libraries
from unsloth import FastLanguageModel, is_bfloat16_supported
import torch
import os
import re
from typing import List, Literal, Optional
from datasets import load_dataset
from trl import KTOConfig, KTOTrainer

# Model Loading and Configuration
Load the pre-trained model and tokenizer using FastLanguageModel, and configure basic parameters like sequence length and quantization settings.

In [ ]:
# Model Loading and Configuration

# Set basic parameters
max_seq_length = 4096  # Choose any! We auto support RoPE Scaling internally!
dtype = None  # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True  # Use 4bit quantization to reduce memory usage. Can be False.


# Load the pre-trained model and tokenizer
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/gpt-oss-20b-unsloth-bnb-4bit",  # Choose ANY! eg mistralai/Mistral-7B-Instruct-v0.2
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
    # token="hf_...",  # use one if using gated models like meta-llama/Llama-2-7b-hf
)

# Add proper chat template if missing
if tokenizer.chat_template is None:
    DEFAULT_CHAT_TEMPLATE = "{% for message in messages %}\n{% if message['role'] == 'user' %}\n{{ '<|user|>\n' + message['content'] + eos_token }}\n{% elif message['role'] == 'system' %}\n{{ '<|system|>\n' + message['content'] + eos_token }}\n{% elif message['role'] == 'assistant' %}\n{{ '<|assistant|>\n'  + message['content'] + eos_token }}\n{% endif %}\n{% if loop.last and add_generation_prompt %}\n{{ '<|assistant|>' }}\n{% endif %}\n{% endfor %}"
    tokenizer.chat_template = DEFAULT_CHAT_TEMPLATE

==((====))==  Unsloth 2024.12.1: Fast Qwen2 patching. Transformers:4.46.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 7.5. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


# Dataset Preparation and Processing
Load and prepare the Claudesidian synthetic dataset from Hugging Face Hub. The dataset contains 1,000 ChatML-formatted examples with boolean labels (true=desirable, false=undesirable). These are converted to KTO's chosen/rejected format and paired for contrastive learning.

In [ ]:
# Dataset Preparation and Processing

# Function to apply chat template
def apply_chat_template(
    example, tokenizer, task: Literal["sft", "generation", "rm", "kto"] = "sft", assistant_prefix="<|assistant|>\n"
):
    def _strip_prefix(s, pattern):
        # Use re.escape to escape any special characters in the pattern
        return re.sub(f"^{re.escape(pattern)}", "", s)

    if task in ["sft", "generation"]:
        messages = example["messages"]
        # We add an empty system message if there is none
        if messages[0]["role"] != "system":
            messages.insert(0, {"role": "system", "content": ""})
        example["text"] = tokenizer.apply_chat_template(
            messages, tokenize=False, add_generation_prompt=True if task == "generation" else False
        )
    elif task == "rm":
        if all(k in example.keys() for k in ("chosen", "rejected")):
            chosen_messages = example["chosen"]
            rejected_messages = example["rejected"]
            # We add an empty system message if there is none
            if chosen_messages[0]["role"] != "system":
                chosen_messages.insert(0, {"role": "system", "content": ""})
            if rejected_messages[0]["role"] != "system":
                rejected_messages.insert(0, {"role": "system", "content": ""})
            example["text_chosen"] = tokenizer.apply_chat_template(chosen_messages, tokenize=False)
            example["text_rejected"] = tokenizer.apply_chat_template(rejected_messages, tokenize=False)
        else:
            raise ValueError(
                f"Could not format example as dialogue for `rm` task! Require `[chosen, rejected]` keys but found {list(example.keys())}"
            )
    elif task == "dpo":
        if all(k in example.keys() for k in ("chosen", "rejected")):
            # Compared to reward modeling, we filter out the prompt, so the text is everything after the last assistant token
            prompt_messages = [[msg for msg in example["chosen"] if msg["role"] == "user"][0]]
            # Insert system message
            if example["chosen"][0]["role"] != "system":
                prompt_messages.insert(0, {"role": "system", "content": ""})
            else:
                prompt_messages.insert(0, example["chosen"][0])
            # TODO: handle case where chosen/rejected also have system messages
            chosen_messages = example["chosen"][1:]
            rejected_messages = example["rejected"][1:]
            example["text_chosen"] = tokenizer.apply_chat_template(chosen_messages, tokenize=False)
            example["text_rejected"] = tokenizer.apply_chat_template(rejected_messages, tokenize=False)
            example["text_prompt"] = tokenizer.apply_chat_template(
                prompt_messages, tokenize=False, add_generation_prompt=True
            )
            example["text_chosen"] = _strip_prefix(example["text_chosen"], assistant_prefix)
            example["text_rejected"] = _strip_prefix(example["text_rejected"], assistant_prefix)
        else:
            raise ValueError(
                f"Could not format example as dialogue for `dpo` task! Require `[chosen, rejected]` keys but found {list(example.keys())}"
            )
    elif task == "kto":
        if all(k in example.keys() for k in ("chosen", "rejected")):
            prompt_messages = [[msg for msg in example["chosen"] if msg["role"] == "user"][0]]
            chosen_messages = prompt_messages + [msg for msg in example["chosen"] if msg["role"] == "assistant"]
            rejected_messages = prompt_messages + [msg for msg in example["rejected"] if msg["role"] == "assistant"]
            if "system" in example:
                chosen_messages.insert(0, {"role": "system", "content": example["system"]})
                rejected_messages.insert(0, {"role": "system", "content": example["system"]})
            example["text_chosen"] = _strip_prefix(tokenizer.apply_chat_template(chosen_messages, tokenize=False), assistant_prefix)
            example["text_rejected"] = _strip_prefix(tokenizer.apply_chat_template(rejected_messages, tokenize=False), assistant_prefix)
        else:
            raise ValueError(f"Could not format example as dialogue for `kto` task!")
    else:
        raise ValueError(
            f"Task {task} not supported, please ensure that the provided task is one of {['sft', 'generation', 'rm', 'dpo', 'kto']}"
        )
    return example

# Load the Claudesidian synthetic dataset from Hugging Face
raw_datasets = load_dataset(
    "professorsynapse/claudesidian-synthetic-dataset",
    data_files="syngen_toolset_v1.0.0_claude.jsonl"
)
train_dataset = raw_datasets["train"]

# Convert the dataset format from conversations/label to chosen/rejected for KTO training
def convert_to_kto_format(example):
    """
    Convert from ChatML conversations format with boolean label to KTO's chosen/rejected format.
    Extract the prompt (user message) to provide explicit structure for KTO trainer.
    
    Input format:
    {
        "conversations": [
            {"role": "user", "content": "..."},
            {"role": "assistant", "content": "..."}
        ],
        "label": true/false
    }
    
    Output format for KTO:
    {
        "prompt": [{"role": "user", "content": "..."}],  # Just the user message
        "chosen": [messages],   # Full conversation with this response
        "rejected": [messages]  # Full conversation with alternative response
    }
    """
    conversations = example["conversations"]
    label = example["label"]
    
    # Extract the prompt (user message) - it's always the first message
    prompt = [msg for msg in conversations if msg["role"] == "user"]
    
    # label=true means this is a desirable example, so it goes to "chosen"
    # label=false means this is an undesirable example, so it goes to "rejected"
    if label:
        return {
            "prompt": prompt,
            "chosen": conversations,
            "rejected": None  # Will be paired later
        }
    else:
        return {
            "prompt": prompt,
            "chosen": None,  # Will be paired later
            "rejected": conversations
        }

# Apply conversion to all examples
converted_dataset = train_dataset.map(convert_to_kto_format)

# Pair desirable and undesirable examples for KTO training
# KTO requires pairs of chosen/rejected examples
desirable_examples = converted_dataset.filter(lambda x: x["chosen"] is not None)
undesirable_examples = converted_dataset.filter(lambda x: x["rejected"] is not None)

# Create pairs by zipping desirable with undesirable
def create_paired_examples(desirable_list, undesirable_list):
    """Create paired examples for KTO training with matching prompts."""
    pairs = []
    # Repeat undesirable examples if fewer than desirable (common case: more desirable than undesirable)
    max_len = max(len(desirable_list), len(undesirable_list))
    
    for i in range(max_len):
        desirable = desirable_list[i % len(desirable_list)]
        undesirable = undesirable_list[i % len(undesirable_list)]
        
        # Use the prompt from the desirable example (they should be semantically similar)
        # Both chosen and rejected should have the same prompt
        pairs.append({
            "prompt": desirable["prompt"],
            "chosen": desirable["chosen"],
            "rejected": undesirable["rejected"]
        })
    
    return pairs

# Get the paired dataset
desirable_list = [example for example in desirable_examples]
undesirable_list = [example for example in undesirable_examples]
paired_examples = create_paired_examples(desirable_list, undesirable_list)

# Convert to HuggingFace dataset
from datasets import Dataset as HFDataset
train_subset = HFDataset.from_dict({
    "prompt": [ex["prompt"] for ex in paired_examples],
    "chosen": [ex["chosen"] for ex in paired_examples],
    "rejected": [ex["rejected"] for ex in paired_examples]
})

# Model Training Setup
Configure the LoRA adapters and set up the KTO trainer with appropriate training arguments. The trainer uses the paired Claudesidian dataset (742 desirable/254 undesirable examples) for contrastive learning.

In [ ]:
# Model Training Setup

# Configure the LoRA adapters
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_alpha=16,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=3407,
)

# Set up the KTO trainer with appropriate training arguments
# Note: We're using the full paired dataset from the Claudesidian synthetic dataset
# Dataset format: {prompt: [user_msg], chosen: [conversation], rejected: [conversation]}
kto_trainer = KTOTrainer(
    model=model,
    args=KTOConfig(
        per_device_train_batch_size=4,
        gradient_accumulation_steps=2,
        num_train_epochs=1,
        learning_rate=5e-7,
        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),
        output_dir="outputs",
        logging_steps=1,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="cosine",
        warmup_ratio=0.1,
        seed=42,
        report_to="none",  # Use this for WandB etc
        remove_unused_columns=False,  # Keep all columns since we use them
    ),
    train_dataset=train_subset,
    processing_class=tokenizer,
)

# Training Execution
Execute the training process with the configured trainer and monitor the training progress.

In [ ]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.748 GB.
1.709 GB of memory reserved.


In [ ]:
kto_trainer.train()

Step,Training Loss
1,0.500000
2,0.500000
3,0.499600
4,0.501600
5,0.499500
6,0.499400
7,0.501200
8,0.499300
9,0.500200
10,0.502200


# Model Saving and Export
Save the trained model in different formats including LoRA adapters and merged model.

In [ ]:
# Model Saving and Export

# Local saving
model.save_pretrained("lora_model")
tokenizer.save_pretrained("lora_model")
# tokenizer.push_to_hub("your_name/lora_model", token = "...") # Online saving

# Save merged model as float16 or int4
if False: # Set to True to save
    model.save_pretrained_merged("merged_model", tokenizer, save_method = "merged_16bit")
    # model.save_pretrained_merged("merged_model", tokenizer, save_method = "merged_4bit")
    # model.save_pretrained_merged("merged_model", tokenizer, save_method = "lora")

# Save to HuggingFace Hub
if False: # Set to True to save
    model.push_to_hub_merged("your_name/model", tokenizer, save_method = "merged_16bit", token = "...")
    # save_method can be "merged_16bit", "merged_4bit", or "lora"

# Save to GGUF format (for llama.cpp)
if False: # Set to True to save
    from transformers import AutoTokenizer
    model.save_pretrained_merged("merged_model", tokenizer, save_method = "merged_16bit")
    !git clone https://github.com/ggerganov/llama.cpp
    !cd llama.cpp && make
    !python3 llama.cpp/convert.py merged_model/ --outfile model-unsloth.gguf
    # Also supports quantization
    !./llama.cpp/quantize model-unsloth.gguf model-unsloth-Q4_K_M.gguf Q4_K_M

Now if you want to load the LoRA adapters we just saved for inference, set `False` to `True`:

In [ ]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "chatml",
    mapping = {"role": "role", "content": "content", "user": "user", "assistant": "assistant"},
)

FastLanguageModel.for_inference(model)

def generate_response(message):
    print("\n" + "="*60 + "\nQUESTION:\n" + "="*60)
    print(message + "\n")
    print("-"*60 + "\nRESPONSE:\n" + "-"*60)

    messages = [{"content": message, "role": "user"}]
    inputs = tokenizer.apply_chat_template(
        messages,
        tokenize = True,
        add_generation_prompt = True,
        return_tensors = "pt"
    ).to("cuda")

    from transformers import TextStreamer
    text_streamer = TextStreamer(tokenizer, skip_special_tokens=True, skip_prompt=True)
    outputs = model.generate(
        input_ids = inputs,
        streamer = text_streamer,
        temperature = 0.1,
        max_new_tokens = 1024,
        use_cache = True
    )
    return outputs

# Test questions - Claudesidian vault operations
questions = [
    # Test 1: Basic content reading scenario
    "I need to review my meeting notes from yesterday. Can you help me find and read the notes?",
    
    # Test 2: Multi-step workflow with workspace context
    "I'm switching to my 'Q4-Planning' workspace. Once switched, create a summary document that lists all my project notes and their status.",
    
    # Test 3: Folder operations and organization
    "My notes are getting disorganized. Rename the 'old-drafts' folder to 'archive-2024' and then create a README.md file inside it explaining its purpose.",
    
    # Test 4: Search and cross-workspace coordination
    "Search across all my workspaces for notes containing 'roadmap' or 'strategy'. After finding them, create a unified index file that links to all results.",
    
    # Test 5: Error handling and recovery
    "I want to create a backup of an important note, but I'm not sure what the exact file path is. Help me find it and then create a backup copy.",
]

# Generate responses
for i, question in enumerate(questions, 1):
    print(f"\n\n{'='*60}\nTEST CASE {i}: Claudesidian Tool Use\n{'='*60}")
    generate_response(question)